In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="PIL")

# Set OpenCV environment variable to suppress warnings
os.environ["OPENCV_IO_PNG_COMPRESSION"] = "0"

# Increase the limit for image size (in pixels) to handle large images
Image.MAX_IMAGE_PIXELS = 300000000

/Users/dalenlim/cs310/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Load category annotations
category_img = pd.read_csv('../data/Category_and_Attribute_Prediction_Benchmark/Anno_coarse/list_category_img.txt', sep='\s+', skiprows=2, header=None)
category_img.columns = ['image_name', 'category_label']

# Load attribute annotations
attr_img = pd.read_csv('../data/Category_and_Attribute_Prediction_Benchmark/Anno_coarse/list_attr_img.txt', sep='\s+', skiprows=2, header=None)
attr_img.columns = ['image_name'] + [f'attr_{i+1}' for i in range(attr_img.shape[1] - 1)]

# Load evaluation partition
eval_partition = pd.read_csv('../data/Category_and_Attribute_Prediction_Benchmark/Eval/list_eval_partition.txt', sep='\s+', skiprows=1, header=None)
eval_partition.columns = ['image_name', 'partition']

# Merge data
df = pd.merge(category_img, eval_partition, on='image_name')
df = pd.merge(df, attr_img, on='image_name')
# Replace "img/" with "img_highres/" in the image paths
df['image_name'] = df['image_name'].apply(lambda x: x.replace('img/', 'img_highres/'))
# print(df.head())

# Split data based on partition
train_df = df[df['partition'] == 'train']
val_df = df[df['partition'] == 'val']
test_df = df[df['partition'] == 'test']

# Downsample to 5% of each category
train_df = train_df.groupby('category_label', group_keys=False).apply(lambda x: x.sample(frac=0.025, random_state=42))
val_df = val_df.groupby('category_label', group_keys=False).apply(lambda x: x.sample(frac=0.025, random_state=42))
test_df = test_df.groupby('category_label', group_keys=False).apply(lambda x: x.sample(frac=0.025, random_state=42))

train_df = train_df.copy()
val_df = val_df.copy()
test_df = test_df.copy()

train_df['category_label'] = train_df['category_label'].astype(str)
val_df['category_label'] = val_df['category_label'].astype(str)
test_df['category_label'] = test_df['category_label'].astype(str)

# Image data generator for augmentation
train_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
val_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

# Training generator
train_data = train_gen.flow_from_dataframe(
    train_df,
    directory='../data/Category_and_Attribute_Prediction_Benchmark/Img/',
    x_col='image_name',
    y_col='category_label',
    target_size=(100, 100),
    batch_size=64,
    class_mode='categorical'
)

# Validation generator
val_data = val_gen.flow_from_dataframe(
    val_df,
    directory='../data/Category_and_Attribute_Prediction_Benchmark/Img/',  
    x_col='image_name',
    y_col='category_label',
    target_size=(100, 100),  # Resize images to 100x100
    batch_size=64,
    class_mode='categorical'
)

/var/folders/j7/7_c1mzmx6jg_btnglq2c7xt00000gn/T/ipykernel_33932/1477337061.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_df = train_df.groupby('category_label', group_keys=False).apply(lambda x: x.sample(frac=0.025, random_state=42))
/var/folders/j7/7_c1mzmx6jg_btnglq2c7xt00000gn/T/ipykernel_33932/1477337061.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  val_df = val_df.groupby('category

Found 5229 validated image filenames belonging to 43 classes.
Found 998 validated image filenames belonging to 34 classes.


In [3]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Load pre-trained Xception model
base_model = Xception(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dense(len(train_data.class_indices), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_data,
    validation_data= val_data,
    epochs=5,
    steps_per_epoch=len(train_data),
    validation_steps=len(val_df) // 64,
    callbacks=[early_stop]
)

2024-11-19 13:37:39.336157: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-11-19 13:37:39.336385: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-11-19 13:37:39.336623: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-11-19 13:37:39.336683: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-19 13:37:39.336912: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/dalenlim/cs310/venv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwar

Epoch 1/5


2024-11-19 13:37:44.302403: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.2775 - loss: 2.7560

ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 34), output.shape=(None, 43)

In [ ]:
test_data = test_gen.flow_from_dataframe(
    test_df,
    directory=img_dir,
    x_col='image_name',
    y_col='category_label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc}")
